## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# since we have 4 cores in our cluster, we set the number of partitions to 4
spark.conf.set('spark.sql.shuffle.partitions', '4')
from pyspark.sql.functions import *
from matplotlib import pyplot as plt
import mlflow
import mlflow.metrics as metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import pandas as pd
from delta.tables import DeltaTable

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE
df1 = get_source_listing_df()
df1

In [0]:
# displaying the total number of files 
files_count = df1.count()
display(files_count)

In [0]:
# showing the contents of a json file
show_s3_file_contents(df1.loc[0]['File Name'])

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Defining the schema for the raw data 
from pyspark.sql import SparkSession 
spark = SparkSession.builder.getOrCreate()

from pyspark.sql.types import StructType, StructField, StringType, TimestampType
schemaBronze = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True),
    StructField("processing_time", TimestampType(), True),
])

display(schemaBronze)


In [0]:
# Setting the bronze stream using cloudfiles and source data format

bronze_stream = (spark
            .readStream
            .format("cloudFiles")
            .option("cloudFiles.format","json")
            .option("mergeSchema",True)
            .schema(schemaBronze)
            .load(TWEET_SOURCE_PATH)
            .withColumn("processing_time", current_timestamp())
            .withColumn("source_file", input_file_name())
            .writeStream
            .format("delta")
            .outputMode("append")
            .option("checkpointLocation", BRONZE_CHECKPOINT)
            .trigger(availableNow=True)
            .start(BRONZE_DELTA)
            )

In [0]:
bronze_stream.awaitTermination()

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# viewing the bronze table
df_b = (spark
            .read
            .format("delta")
            .load(BRONZE_DELTA))

display(df_b)

In [0]:
# getting the number of tweets
tweets = df_b.count()
print("No. of tweets:",tweets)

In [0]:
# getting total nulls 
nulls = []
for col_name in df_b.columns:
    null_count = df_b.filter(col(col_name).isNull()).count()
    nulls.append((col_name, null_count))
    
for col_name, null_count in nulls:
    print(col_name, null_count)


In [0]:
# getting total nans
nans = []
for col_name in df_b.columns:
    if col_name != 'processing_time':   # we do not check for 'processing_time' is it is a timestamp and cannot be NaN
        nan_count = df_b.filter(isnan(col(col_name))).count()
        nans.append((col_name, nan_count))
    
    
for col_name, nan_count in nans:
    print(col_name, nan_count)

In [0]:
# Counting the number of tweets by each unique user handle and sort the data by descending count
tweets_user = (df_b
                .groupBy("user").count()
                .sort(desc("count")))

display(tweets_user)

In [0]:
# Counts of tweets with at least one mention and with no mentions

mentions_df = df_b.withColumn("mentions", when(col("text").contains("@"), True).otherwise(False))
mentions_count = mentions_df.filter(col("mentions") == True).count()
no_mentions_count = mentions_df.filter(col("mentions") == False).count()

print("No. of tweets with at least one mention",mentions_count)
print("No. of tweets with no mentions",no_mentions_count)

In [0]:
# bar chart for top 20 tweeters

pandas_tweet_user = tweets_user.limit(20).toPandas() # conversion for using pyplot

plt.figure(figsize=(10, 6))
plt.bar(pandas_tweet_user['user'], pandas_tweet_user['count'], color='green')
plt.xlabel('USER')
plt.ylabel('TWEETS')
plt.title('Bar chart for tweet counts of top 20 tweeters')
plt.xticks(rotation=90)
plt.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# reading from the bronze table
read_bronze = (
    spark.readStream
    .format('delta')
    .load(BRONZE_DELTA)
)

In [0]:
# bronze to silver conversions

# needs to be set for timestamp conversion to work
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
df_bronze_to_silver = read_bronze.withColumn("timestamp", to_timestamp("date", "E MMM dd HH:mm:ss z yyyy"))
df_bronze_to_silver = df_bronze_to_silver.withColumn("all_mentions", expr("regexp_extract_all(text, '(?<=^|(?<=[^a-zA-Z0-9-_\\\\.]))@([A-Za-z]+[A-Za-z0-9_]+)', 0)"))
df_bronze_to_silver = df_bronze_to_silver.withColumn("mention", explode(col("all_mentions"))).drop("all_mentions")
df_bronze_to_silver = df_bronze_to_silver.withColumn("cleaned_text", regexp_replace("text", "(@\\w+)", ""))
df_bronze_to_silver = df_bronze_to_silver.drop("date", "user", "text", "source_file", "processing_time")
df_bronze_to_silver

In [0]:
# starting the stream for silver table
silver_stream = (df_bronze_to_silver.writeStream 
    .format("delta") 
    .outputMode("append") 
    .option("checkpointLocation", SILVER_CHECKPOINT) 
    .trigger(availableNow=True)
    .queryName('silver_stream') 
    .start(SILVER_DELTA))

In [0]:
silver_stream.awaitTermination()

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# starting the read for silver
silver_read = (
    spark.readStream
    .format('delta')
    .load(SILVER_DELTA)
)

In [0]:
# load model
model_uri_path = f"models:/{MODEL_NAME}/Production"
model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri_path)

# conversion from silver to gold 
df_silver_to_gold = silver_read.withColumn("temp_predicted_sentiment", model(col("cleaned_text")))
df_silver_to_gold = df_silver_to_gold.withColumn("predicted_score", (col("temp_predicted_sentiment.score") * 100).cast("int"))
df_silver_to_gold = df_silver_to_gold.withColumn("predicted_sentiment", col("temp_predicted_sentiment.label"))
df_silver_to_gold = df_silver_to_gold.withColumn("sentiment_id", when(col("sentiment") == "positive", 1).otherwise(0))
df_silver_to_gold = df_silver_to_gold.withColumn("predicted_sentiment_id", when(col("predicted_sentiment") == "POS", 1).otherwise(0))

df_silver_to_gold = df_silver_to_gold.drop("temp_predicted_sentiment")

In [0]:
# starting the stream for gold table
gold_stream = (
    df_silver_to_gold.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .trigger(availableNow=True)
    .queryName("gold_stream")
    .start(GOLD_DELTA)
)

In [0]:
gold_stream.awaitTermination()

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# starting the read for gold
gold_read = (
    spark.read
    .format('delta')
    .load(GOLD_DELTA)
)

In [0]:
# fetching the metrics, etc. and storing within an ML flow experiment run
 
with mlflow.start_run() as run:

    pandas_df = gold_read.toPandas()

    # sentiment_true = gold_read.select(col('sentiment_id')).collect()
    # sentiment_pred = gold_read.select(col('predicted_sentiment_id')).collect()

    true_labels = pandas_df['sentiment_id']
    pred_labels = pandas_df['predicted_sentiment_id']

    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)

    # logging the precision, recall and f1_score within the run
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    cmx = confusion_matrix(true_labels, pred_labels)

    fig, ax = plt.subplots(figsize=(8,8))
    disp = ConfusionMatrixDisplay(confusion_matrix=cmx)
    disp.plot(cmap=plt.cm.Blues, ax=ax)
    plt.savefig("cmx.png")
    plt.show()

    # storing the confusion matrix as an ML artifact
    mlflow.log_artifact("cmx.png")

    # storing the model_name and mlflow_version as MLFlow parameters
    mlflow.log_param("model_name",  MODEL_NAME)
    mlflow.log_param("mlflow_version", mlflow.__version__)

    delta_history = DeltaTable.forPath(spark, SILVER_DELTA).history() \
    .select("version") \
    .orderBy("version", ascending=False)

    delta_version = delta_history.collect()[0][0]
    # storing the delta version for Silver
    mlflow.log_param("delta_table_version", delta_version)


## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# getting the total number of mentions
mentions_in_gold = gold_read.filter(col("mention").isNotNull()).count()
print("Number of total mentions in gold data: ",mentions_in_gold)

# storing the counts of neutral, positive and negative tweets for each mention in df_agg
df_agg = gold_read.groupBy("mention").agg(
    count(when(col("sentiment") == "neutral", True)).alias("neutral_count"),
    count(when(col("sentiment") == "positive", True)).alias("positive_count"),
    count(when(col("sentiment") == "negative", True)).alias("negative_count"),
    count("mention").alias("total_mentions") # capturing total for each mention
).orderBy("total_mentions", ascending=False)


# getting top 20 mentions of positive and negative sentiments
top_positive_mentions = df_agg.orderBy("positive_count", ascending=False).limit(20)
top_negative_mentions = df_agg.orderBy("negative_count", ascending=False).limit(20)


def plot_mentions(df, title, sentiment):
    data = df.select("mention", sentiment).collect()
    mentions = [x["mention"] for x in data]
    counts = [x[sentiment] for x in data]
    
    plt.figure(figsize=(10, 5))
    plt.bar(mentions, counts, color='blue')
    plt.xlabel('Mentions')
    plt.ylabel('Counts')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Plot for top 20 positive mentions
plot_mentions(top_positive_mentions, 'Top 20 Mentions with Positive Sentiment', 'positive_count')

# Plot for top 20 negative mentions
plot_mentions(top_negative_mentions, 'Top 20 Mentions with Negative Sentiment', 'negative_count')

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# # running streams
for stream in spark.streams.active:
    print(stream.name)

# # Stop all active streams
stop_all_streams()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

elapsed_time = END_TIME - START_TIME
minutes = int(elapsed_time // 60)
seconds = elapsed_time % 60
print(f"Complete elasped time: {minutes} min, {seconds:.2f} sec")

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE

Spill:
The system shows no signs of data spill, evidenced by even partition sizes and small source files.
Absence of large files prevents issues like uneven load and processing, avoiding performance problems from disk spills.

Skew:
Data is evenly distributed in job executions, helped by uniform file sizes and Adaptive Query Execution optimizing partitions.
This even distribution means no single node is overwhelmed, preventing bottlenecks.

Shuffle:
Shuffle operations are common, needed for tasks like GroupBy and OrderBy.
Adjusting the number of shuffle partitions to match the number of cores helps optimize data movement and reduces shuffle delays.
Implication: Strategic shuffle configuration minimizes the time delays in these necessary operations.

Storage:
Storing many small JSON files in S3 initially leads to slow read times.
After the initial read, data is stored more efficiently in Bronze, Silver, and Gold Delta tables. Focus on improving the initial file handling by possibly consolidating files or tweaking the read process.

Serialization:
Serializing the ML model for sentiment analysis as a UDF takes significant time.
While replacing the UDF isn't an option, optimizing other UDFs and considering built-in Spark functions could improve efficiency and reduce dependence on slower custom UDFs.